# Enzyme Constrained Flux Balance Analysis

## Introduction

In this short tutorial, we demonstrate how to use the model variant EC-*i*CH360 to run enzyme-constrained flux balance analysis (EC-FBA) and generate predictions of enzyme allocation across the network. EC-FBA is an extension of traditional Flux Balance Analysis that accounts, even if in a simplified way, for the enzymatic resources that must be allocated to support a given flux distribution through the metabolic network. 

This is achieved in EC-*i*CH360 by assigning each enzymatic reaction an enzyme cost $a_i$ (in g $\cdot$ h/mmol) such that the enzyme mass (in g/gDW) that must be allocated to support a flux $v_i$ (in mmol/gDW*h) through the reaction is given by:
$$
e_i=a_i~v_i
$$
Hence, the EC model limits the space of possible flux distributions by assuming a maximal enzyme budget $E_{\rm tot}$ (in g/gDW) that the cell can allocate to metabolism. This corresponds to adding the the following (linear) constraint in the FBA problem:
$$
\underbrace{\sum_i~a_i~v_i}_{\rm total~enzyme~cost} \leq E_{\rm tot}
$$
Note that, in practice, $E_{\rm tot}$ can be interpreted as the fraction of biomass that metabolic enzymes are allowed to occupy. 

## Loading the required packages
EC-*i*CH360 can be loaded and simulated with the COBRA toolbox, so we won't require other metabolic modelling dependencies for this tutorial.

In [125]:
import cobra        #for model analysis
import pandas as pd #to post-process results

## Reading the EC model
EC-*i*CH360 is a COBRA model just like any other, so it can be read using the standard COBRA I/O functions:

In [126]:
EC_iCH360=cobra.io.read_sbml_model('../Model/EC_iCH360/EC_iCH360.xml')
ec_model

Name,EC_iCH360
Memory address,181a5d89ff0
Number of metabolites,305
Number of reactions,505
Number of genes,356
Number of groups,0
Objective expression,1.0*Biomass_fw - 1.0*Biomass_fw_reverse_bf19d
Compartments,"cytosol, extracellular space, periplasm"


As you can see, EC-*i*CH360 shares the same number of metabolites and genes as *i*CH360 but appears to have many more (almost twice as many) reactions as the standard model. The reason for this is that to enforce the enzyme resources constraint described in the [Introduction](#introduction), all fluxes are required to be positive (else you could have a negative enzyme cost for a reaction operating backward!). Hence, like many other enzyme-constrained model, reversible reactions in EC-*i*CH360 are split in forward and backwards components. Forward and backwards components of reactions are flagged by adding a `_fw`, and `_bw` suffix, respectively, to the reaction ID. For example, reaction `GAPD` appears as:

In [127]:
print(EC_iCH360.reactions.get_by_id('GAPD_fw')) #forward component
print(EC_iCH360.reactions.get_by_id('GAPD_bw')) #forward component) #backward component

GAPD_fw: 0.000672197517699226 enzyme_pool + g3p_c + nad_c + pi_c --> 13dpg_c + h_c + nadh_c
GAPD_bw: 13dpg_c + 0.0026437278052183 enzyme_pool + h_c + nadh_c --> g3p_c + nad_c + pi_c


Note how the stoichiometry of the metabolites (ignoring the `enzyme_pool` item for now ) is inverted in the two reactions. Also, for consistency in the naming, the `_fw` suffix is also used used for irreversible reaction, which don't have a corresponding backward component:

In [128]:
print(EC_iCH360.reactions.get_by_id('PDH_fw'))
print(EC_iCH360.reactions.get_by_id('PDH_bw') if 'PDH_bw' in EC_iCH360.reactions else "PDH_bw not in the model!")

PDH_fw: coa_c + 0.00128810216333717 enzyme_pool + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
PDH_bw not in the model!


## The sMOMENT format

EC-*i*CH360 was constructed using the sMOMENT format [1]. This is a particularly simple way to enhance a standard metabolic model with enzyme constraints and (in short) works as follows:
1. Every reaction (after splitting in forward and backwards components) is assigned a unique enzyme.
2. An "enzyme pool" pseudometabolite is introduced in the model. This pseudometabolite is consumed by each enzymatic reaction with stoichiometry corresponding to the enzyme cost (per unit flux) of that reaction, i.e. $a_i$ in [Introduction](#introduction).
3. An "enzyme supply" pseudoreaction is introduced as a source of enzyme pool. The flux through this reaction is bounded by the allowable enzyme budget, i.e. $E_{\rm tot}$ in [Introduction](#introduction).

That's it! You may want to convince yourself that step 2 and 3 above correspond exactly to bounding the total enzyme cost in the model by 
$E_{\rm tot}$, as we described in the in [Introduction](#introduction)..

 Compared to other formats, such as GECKO [2], this formulation imposes some limitations on the possible problem formulations supported by the EC-model (for example, it doesn't allow for individual bounds on each enzyme), but comes with the great advantage of only requiring one extra (pseudo)reaction and (pseudo)metabolite to be added to the model. In EC-*i*CH360, the enzyme pool pseudometabolite and the enzyme supply pseudoreactions are identified by the `enzyme_pool` and `enzyme_pool_supply` IDs:

In [129]:
print(EC_iCH360.metabolites.get_by_id('enzyme_pool'))
print(EC_iCH360.reactions.get_by_id('enzyme_pool_supply'))

enzyme_pool
enzyme_pool_supply:  --> enzyme_pool


As we already saw before, the enzyme pool metabolite is consumed by all enzymatic reactions:

In [130]:
enzyme_pool=EC_iCH360.metabolites.get_by_id('enzyme_pool')
print(f"{enzyme_pool} is consumed by ",
      len([r for r in EC_iCH360.reactions if enzyme_pool in r.metabolites]),
      " reactions"
      )

enzyme_pool is consumed by  428  reactions


## Enzyme constrained flux balance analysis

To simulate EC-*i*CH360, we simply need to run normal flux balance analysis on it (as per usual, EC-*i*CH360 is configured by default for aerobic growth on glucose, using maximisation of growth rate as an objective). The addition of the enzyme pool/supply metabolite and reactions automatically introduce the enzyme constraint in the problem. However, there is a fundamental difference with standard Flux Balance Analysis. In standard FBA, we *must* bound carbon source uptake, else, the problem becomes unbounded. in EC-FBA, carbon uptake needs not to be bounded, as the the enzyme budget constraint will inevitably be limiting at some point:

In [131]:
print(f"Glucose uptake bound: {EC_iCH360.reactions.get_by_id('EX_glc__D_e_bw').upper_bound}")
ec_fba_sol=EC_iCH360.optimize()
print(f"EC-FBA objective: {round(ec_fba_sol.fluxes['Biomass_fw'],2)}/h")
ec_fba_sol.fluxes.head()

Glucose uptake bound: 1000.0
EC-FBA objective: 0.62/h


NDPK5_fw     0.016660
SHK3Dr_fw    0.234334
NDPK6_fw     0.016136
NDPK8_fw     0.016136
DHORTS_fw    0.000000
Name: fluxes, dtype: float64

That was easy! As per usual, we can change the growth conditions by modifying the allowable exchanges (remember that, by convention, exchange fluxes are negative in the uptake direction, so that we need to modify bounds on the `_bw` components to modulate uptake):

In [132]:
#Anaerobic growth on fructose

with EC_iCH360 as ec_model:
    #Block uptake of glucose
    ec_model.reactions.get_by_id('EX_glc__D_e_bw').upper_bound=0
    #Allow uptake of fructose
    ec_model.reactions.get_by_id('EX_fru_e_bw').upper_bound=1000
    #Block uptake of oxygen
    ec_model.reactions.get_by_id('EX_o2_e_bw').upper_bound=0.

    #Run EC-FBA
    ec_fba_sol_fru_anaer=EC_iCH360.optimize()
    print(f"EC-FBA objective: {round(ec_fba_sol_fru_anaer.fluxes['Biomass_fw'],2)}/h")
    ec_fba_sol_fru_anaer.fluxes.head()


EC-FBA objective: 0.11/h


## Converting directional fluxes into net fluxes

As we've seen, running EC-FBA on EC-*i*CH360 does not require any extra steps than running FBA on the stoichiometric model. However, the resulting flux distribution are given in terms of directional fluxes (forward and backwards) components, which are not *directly* comparable with those of the normal model (and cannot, for example, be used for visualisation with the metabolic maps).

To obtain the net fluxes, we simply subtract the forward and backward component of each reaction, paying attention that not all reactions have a backward components:

In [133]:
fluxes=ec_fba_sol.fluxes
forward_reaction_ids=[r.id for r in EC_iCH360.reactions if '_fw' in r.id]
net_fluxes=pd.Series(dtype=float)

for fw_rxn_id in forward_reaction_ids:
    #Remove the direction suffix
    net_reaction_id=fw_rxn_id.replace('_fw','')
    #If the reaction has a backward component, subtract it from the forward one to obtain net flux
    bw_rxn_id=fw_rxn_id.replace('_fw','_bw')
    if  bw_rxn_id in EC_iCH360.reactions:
        net_fluxes[net_reaction_id]=fluxes[fw_rxn_id]-fluxes[bw_rxn_id]
    else:
        net_fluxes[net_reaction_id]=fluxes[fw_rxn_id]
net_fluxes.head()

NDPK5     0.016660
SHK3Dr    0.234334
NDPK6     0.016136
NDPK8     0.016136
DHORTS   -0.203989
dtype: float64

For those that like condensed syntax, the following pythonic one-liner achieves the same as the above loop.

In [134]:
net_fluxes=pd.Series({fw_rxn_id.replace('_fw',''):  fluxes[fw_rxn_id]-fluxes[fw_rxn_id.replace('_fw','_bw')]  
                                                    if fw_rxn_id.replace('_fw','_bw') in EC_iCH360.reactions 
                                                    else fluxes[fw_rxn_id]
                                                    for fw_rxn_id in forward_reaction_ids
                      })
net_fluxes.head()

NDPK5     0.016660
SHK3Dr    0.234334
NDPK6     0.016136
NDPK8     0.016136
DHORTS   -0.203989
dtype: float64

## Extracting enzyme abundances

While EC-FBA formally optimised fluxes, it implicitly generates an  optimal allocation of metabolic enzyme across its network, based on the enzyme cost that parametrise the model. EC-*i*CH360 does not include enzymes as explicit variables (which would require introduction of many pseudoreactions/metabolites in the model), so we need to "extract" this information from the solution. To enable this to be done without the need of external tables/mappings, we added the relevant information as annotations in each reaction.

For example, each enzymatic reactions contains an annotation field `smoment_enzyme`, which contains the enzyme mapped to this reaction when constructing the sMOMENT:

In [135]:
EC_iCH360.reactions.get_by_id('CS_fw').annotation['smoment_enzyme']

'CITRATE-SI-SYNTHASE'

Each enzyme in  EC-*i*CH360 maps to a protein in the [knowledge graph](../Docs/knowledge_graph.md), which can therefore be used to extract all relevant information (and external annotations) on the enzyme. You can find more information on how a single enzyme was assigned to each reaction in the manuscript supporting the model. The predicted abundance of an enzyme can therefore be computed by summing all the contributions from different reactions that use the enzyme. Remember that the contribution of each reaction to the total enzyme pool is computed as $a_i~v_i$ where the cost coefficient $a_i$ is the (negative) stoichiometry with which the enzyme pool pseudometabolite is consumed in each reaction

The following code achieves exactly this, using the fluxes computed in the previous section


In [136]:
# Let's extract the ID of all enzymes in the model
enzyme_ids=set([r.annotation['smoment_enzyme'] for r in EC_iCH360.reactions if 'smoment_enzyme' in r.annotation])
#Initialise a Series that will contain the predicted abundance of each enzyme (for now, set abundance to 0. for all enzymes)
enzyme_abundances=pd.Series(index=enzyme_ids,data=0.,name='enzyme_abundance_g_gDW')

enzyme_pool=EC_iCH360.metabolites.get_by_id("enzyme_pool") #we will use this to retrieve the enzyme cost of each reaction
for r in EC_iCH360.reactions: 
     #If the reaction consumes enzymatic resources, compute its contribution and add that to the enzyme abundanc evector
     if enzyme_pool in r.metabolites and r.id!='enzyme_pool_supply':
        flux=fluxes[r.id] #flux of the reaction in the EC-FBA solution
        enzyme_cost=-r.metabolites[enzyme_pool] #enzyme cost per unit flux
        enzyme_id=r.annotation['smoment_enzyme'] #enzyme used by the reaction
        enzyme_abundances[enzyme_id]+=enzyme_cost*flux #enzyme abundance in g/gDW

enzyme_abundances.head()

TRANSKETOI-CPLX         0.004065
CPLX0-8005              0.001482
ADENYLYLSULFKIN-CPLX    0.000084
PURE-CPLX               0.000315
ABC-12-CPLX             0.000000
Name: enzyme_abundance_g_gDW, dtype: float64

We can verify that the total predicted abundance matches exactly the enzyme budget in the model (the upper bound on the `enzyme_pool_supply` reaction);

In [137]:
enzyme_abundances.sum()==EC_iCH360.reactions.get_by_id('enzyme_pool_supply').upper_bound

True

## Putting all together

The following code merges summarises what we've shown in the previous sections and can be used to merge the EC-FBA solution (net fluxes and enzyme abundances) into a single pandas DataFrame

In [138]:
def EC_FBA(ec_model):
    # First, optimise the model and extract optimal fluxes
    ec_fba_sol=ec_model.optimize()
    fluxes=ec_fba_sol.fluxes
    #====================================
    #Compute net fluxes from the solution
    forward_reaction_ids=[r.id for r in ec_model.reactions if '_fw' in r.id]
    net_fluxes={fw_rxn_id.replace('_fw',''):  fluxes[fw_rxn_id]-fluxes[fw_rxn_id.replace('_fw','_bw')]  
                                                    if fw_rxn_id.replace('_fw','_bw') in ec_model.reactions 
                                                    else fluxes[fw_rxn_id]
                                                    for fw_rxn_id in forward_reaction_ids
                      }
    #========================================
    #Compute enzyme abundances from the solution
    enzyme_ids=set([r.annotation['smoment_enzyme'] for r in ec_model.reactions if 'smoment_enzyme' in r.annotation])
    #Initialise a Series that will contain the predicted abundance of each enzyme (for now, set abundance to 0. for all enzymes)
    enzyme_abundances=pd.Series(index=enzyme_ids,data=0.,name='enzyme_abundance_g_gDW')

    enzyme_pool=ec_model.metabolites.get_by_id("enzyme_pool") #we will use this to retrieve the enzyme cost of each reaction
    for r in ec_model.reactions: 
        #If the reaction consumes enzymatic resources, compute its contribution and add that to the enzyme abundanc evector
        if enzyme_pool in r.metabolites and r.id!='enzyme_pool_supply':
            flux=fluxes[r.id] #flux of the reaction in the EC-FBA solution
            enzyme_cost=-r.metabolites[enzyme_pool] #enzyme cost per unit flux
            enzyme_id=r.annotation['smoment_enzyme'] #enzyme used by the reaction
            enzyme_abundances[enzyme_id]+=enzyme_cost*flux #enzyme abundance in g/gDW
    #=======================================================
    #Create an output table, which merge reactions and enzyme abundance
    out=pd.DataFrame(index=net_fluxes.keys(),columns=['flux','enzyme','enzyme_abundance_g_gDW'])
    for r_id,flux in net_fluxes.items():
        #Since all reactions in EC-iCH360 have a forward component, but not all have a backwards one, we will use the forward to retrieve the enzyme mapped to the reaction
        forward_reaction=ec_model.reactions.get_by_id(r_id+'_fw')
        enzyme_id=forward_reaction.annotation['smoment_enzyme'] if 'smoment_enzyme' in forward_reaction.annotation else 'NA'
        out.loc[r_id,'flux']=flux
        if enzyme_id in enzyme_abundances.index:
            #If we do have an enzyme for this reaction, store that in the table
            out.loc[r_id,'enzyme']=enzyme_id
            out.loc[r_id,'enzyme_abundance_g_gDW']=enzyme_abundances[enzyme_id]
    return out

Here it is in action:

In [139]:
ec_fba_results=EC_FBA(EC_iCH360)
ec_fba_results.head()

,flux,enzyme,enzyme_abundance_g_gDW
NDPK5,0.01666,NUCLEOSIDE-DIP-KIN-CPLX,0.002093
SHK3Dr,0.234334,AROE-MONOMER,0.000036
NDPK6,0.016136,NUCLEOSIDE-DIP-KIN-CPLX,0.002093
NDPK8,0.016136,NUCLEOSIDE-DIP-KIN-CPLX,0.002093
DHORTS,-0.203989,DIHYDROOROT-CPLX,0.001043


## Modifying model parameters
In this tutorial, we explored how to use EC-*i*CH360 with its default parameters. If you wish to change some of the model parametrisation, here are some hints:
### Using different turnover parameters
If you wish to use your own turnover parameters for one or multiple reactions, you'll first have to compute an enzyme cost (enzyme mass required per unit flux) in units of g*h/mmol. Typically in enzyme constraint models these are computed as 
$$
a_i=\frac{M_i}{\sigma~k_{\rm cat,i}}
$$
where $M_i$ is the molecular weight of the enzyme (in kDa), $k_{\rm cat,i}$ is the turnover parameter for the enzyme-reaction pair, and $\sigma$ is a scaling accounting e.g. for saturation effects. In practice, it doesn't matter how you choose to compute the enzyme cost, as only the final value is used as a parameter in EC-*i*CH360. You can retrieve the molecular weight ($M_i$) for the enzyme in EC-*i*CH360 via the annotation field `smoment_mw`, or you can use the knowledge graph if you wish to choose a different enzyme for that reaction!

Having computed your own flux cost(s), you simply need to change them in the model by changing the stoichiometry with which the `enzyme_pool` metabolite is consumed by an enzymatic reaction (pay attention to the minus sign, as the enzyme pool is always consumed by metabolic reactions!). For example, to change the flux cost of the`GAPD_fw`, you can run:

In [140]:
modified_EC_iCH360=EC_iCH360.copy() #make a copy of the model
enzyme_pool=modified_EC_iCH360.metabolites.get_by_id('enzyme_pool') #identify the enzyme pool metabolite
new_flux_cost=0.001 #g*h/mmol
#Update the cost. Using combine=False ensure the old cost is overriden
modified_EC_iCH360.reactions.get_by_id('GAPD_fw').add_metabolites({enzyme_pool: -new_flux_cost},combine=False)
print(f"\nNew flux cost: {-modified_EC_iCH360.reactions.get_by_id('GAPD_fw').metabolites[enzyme_pool]} g*h/mmol")

Read LP format model from file C:\Users\marco\AppData\Local\Temp\tmp54fwvkfy.lp
Reading time = 0.01 seconds
: 305 rows, 1010 columns, 4816 nonzeros

New flux cost: 0.001 g*h/mmol


### Changing the enzyme mass budget
This can be achieved by changing the upper bound of the `enzyme_pool_supply` reaction:

In [141]:
new_enzyme_budget=0.32 #in g/gDW
modified_EC_iCH360.reactions.get_by_id('enzyme_pool_supply').upper_bound=new_enzyme_budget

## References
1. Bekiaris, P. S. & Klamt, S. Automatic construction of metabolic models with enzyme constraints. BMC Bioinformatics 21, 19 (2020).